Basic model

In [3]:
import torch.nn as nn
model=nn.Sequential(
            nn.Linear(in_features=784,out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256,out_features=10),
            nn.LogSoftmax(dim=1)
        )
import torch.optim as optim
optimizer=optim.SGD(
    model.parameters(),
    lr=0.01
)
criterion = nn.CrossEntropyLoss()

Load Dataset

In [16]:
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)
batch_size=64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Training

In [23]:
num_epochs = 25

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.view(images.size(0), -1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/25], Loss: 0.1125
Epoch [2/25], Loss: 0.1077
Epoch [3/25], Loss: 0.1035
Epoch [4/25], Loss: 0.0995
Epoch [5/25], Loss: 0.0960
Epoch [6/25], Loss: 0.0925
Epoch [7/25], Loss: 0.0893
Epoch [8/25], Loss: 0.0866
Epoch [9/25], Loss: 0.0835
Epoch [10/25], Loss: 0.0810
Epoch [11/25], Loss: 0.0784
Epoch [12/25], Loss: 0.0761
Epoch [13/25], Loss: 0.0739
Epoch [14/25], Loss: 0.0717
Epoch [15/25], Loss: 0.0697
Epoch [16/25], Loss: 0.0676
Epoch [17/25], Loss: 0.0659
Epoch [18/25], Loss: 0.0641
Epoch [19/25], Loss: 0.0626
Epoch [20/25], Loss: 0.0607
Epoch [21/25], Loss: 0.0595
Epoch [22/25], Loss: 0.0581
Epoch [23/25], Loss: 0.0564
Epoch [24/25], Loss: 0.0550
Epoch [25/25], Loss: 0.0540


Testing

In [24]:
import torch
model.eval()
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for not remebering history
    for images, labels in test_loader:
        images = images.view(images.size(0), -1)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

Accuracy of the model on the test images: 97.74%


saving model

In [25]:
torch.save(model.state_dict(), 'mnist.pth')